<a href="https://colab.research.google.com/github/PedroRainville/MarketPulse-BR/blob/main/brazilian_e_commerce_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
clientes = pd.read_csv('/content/drive/MyDrive/Brazilian-e-commerce-dt/olist_customers_dataset.csv')
pedidos = pd.read_csv('/content/drive/MyDrive/Brazilian-e-commerce-dt/olist_orders_dataset.csv')
itens_do_pedido = pd.read_csv('/content/drive/MyDrive/Brazilian-e-commerce-dt/olist_order_items_dataset.csv')
produtos = pd.read_csv('/content/drive/MyDrive/Brazilian-e-commerce-dt/olist_products_dataset.csv')
reviews = pd.read_csv('/content/drive/MyDrive/Brazilian-e-commerce-dt/olist_order_reviews_dataset.csv')


Importanto e carregando arquivos

In [ ]:
clientes.head()
pedidos.info()
produtos.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
count,32341.000000,32341.000000,32341.000000,32949.000000,32949.000000,32949.000000,32949.000000
mean,48.476949,771.495285,2.188986,2276.472488,30.815078,16.937661,23.196728
std,10.245741,635.115225,1.736766,4282.038731,16.914458,13.637554,12.079047
min,5.000000,4.000000,1.000000,0.000000,7.000000,2.000000,6.000000
25%,42.000000,339.000000,1.000000,300.000000,18.000000,8.000000,15.000000
50%,51.000000,595.000000,1.000000,700.000000,25.000000,13.000000,20.000000
75%,57.000000,972.000000,3.000000,1900.000000,38.000000,21.000000,30.000000
max,76.000000,3992.000000,20.000000,40425.000000,105.000000,105.000000,118.000000


Análise exploratória para entendermos os tipos de dados e suas anomalias

In [ ]:
pedidos.drop_duplicates(inplace=True)

Limpeza de dados Removendo duplicados

In [ ]:
pedidos['order_purchase_timestamp'] = pd.to_datetime(pedidos['order_purchase_timestamp'])

Tratando valores nulos e convertendo datas

In [ ]:
df = pedidos.merge(clientes, on="customer_id").merge(itens_do_pedido, on="order_id")

Conectando as tabelas e colunas

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

pio.renderers.default = 'colab'

pedidos['order_approved_at'] = pd.to_datetime(pedidos['order_approved_at'], errors='coerce')
pedidos['order_delivered_carrier_date'] = pd.to_datetime(pedidos['order_delivered_carrier_date'], errors='coerce')
pedidos['order_delivered_customer_date'] = pd.to_datetime(pedidos['order_delivered_customer_date'], errors='coerce')
pedidos['order_estimated_delivery_date'] = pd.to_datetime(pedidos['order_estimated_delivery_date'], errors='coerce')

print("✅ Dados preparados!")
pedidos.info()

✅ Dados preparados!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 6.1+ MB


In [ ]:
df_revenue_month = itens_do_pedido.merge(
    pedidos[['order_id', 'order_purchase_timestamp']],
    on='order_id'
)
df_revenue_month['revenue'] = df_revenue_month['price'] + df_revenue_month['freight_value']
df_revenue_month['ano_mes'] = df_revenue_month['order_purchase_timestamp'].dt.to_period('M').astype(str)

revenue_month = df_revenue_month.groupby('ano_mes')['revenue'].sum().reset_index()
revenue_month['ano_mes'] = pd.to_datetime(revenue_month['ano_mes'])
revenue_month = revenue_month.sort_values('ano_mes')

revenue_month['ano_mes'] = revenue_month['ano_mes'].dt.strftime('%m/%Y')

display(revenue_month.head(12).style.format({'revenue': 'R$ {:.2f}'}))

fig = px.line(revenue_month, x='ano_mes', y='revenue',
              title='Revenue por Mês', markers=True)
fig.update_yaxes(title='Revenue (R$)', tickformat='R$ ')
fig.show()

revenue_month.to_csv('revenue_por_mes.csv', index=False)

,ano_mes,revenue
0,09/2016,R$ 354.75
1,10/2016,R$ 56808.84
2,12/2016,R$ 19.62
3,01/2017,R$ 137188.49
4,02/2017,R$ 286280.62
5,03/2017,R$ 432048.59
6,04/2017,R$ 412422.24
7,05/2017,R$ 586190.95
8,06/2017,R$ 502963.04
9,07/2017,R$ 584971.62


Revenue por mes de cada ano

In [ ]:
df_revenue_month = itens_do_pedido.merge(
    pedidos[['order_id', 'order_purchase_timestamp']],
    on='order_id'
)
df_revenue_month['revenue'] = df_revenue_month['price'] + df_revenue_month['freight_value']
df_revenue_month['ano_mes'] = df_revenue_month['order_purchase_timestamp'].dt.to_period('M').astype(str)

df_revenue_cat = df_revenue_month.merge(
    produtos[['product_id', 'product_category_name']],
    on='product_id', how='left'
).fillna({'product_category_name': 'Sem Categoria'})

revenue_cat = df_revenue_cat.groupby('product_category_name')['revenue'].sum().reset_index()
revenue_cat = revenue_cat.sort_values('revenue', ascending=False).head(10)

display(revenue_cat.style.format({'revenue': 'R$ {:.2f}'}))
fig = px.bar(revenue_cat, x='revenue', y='product_category_name',
             title='Top 10 Revenue por Categoria', orientation='h')
fig.update_layout(xaxis_title='Revenue (R$)', xaxis_tickformat='R$ ')
fig.show()

revenue_cat.to_csv('revenue_por_categoria.csv', index=False)

,product_category_name,revenue
12,beleza_saude,R$ 1441248.07
67,relogios_presentes,R$ 1305541.61
14,cama_mesa_banho,R$ 1241681.72
33,esporte_lazer,R$ 1156656.48
45,informatica_acessorios,R$ 1059272.40
55,moveis_decoracao,R$ 902511.79
73,utilidades_domesticas,R$ 778397.77
27,cool_stuff,R$ 719329.95
9,automotivo,R$ 685384.32
41,ferramentas_jardim,R$ 584219.21


Revenue por categoria Top 10 categorias com maior revenue

In [ ]:
revenue_seller = df_revenue_month.groupby('seller_id')['revenue'].sum().reset_index()
revenue_seller = revenue_seller.sort_values('revenue', ascending=False).head(10)

display(revenue_seller.style.format({'revenue': 'R$ {:.2f}'}))
fig = px.bar(revenue_seller, x='revenue', y='seller_id',
             title='Top 10 Vendedores por Revenue', orientation='h')
fig.show()

revenue_seller.to_csv('revenue_por_vendedor.csv', index=False)

,seller_id,revenue
857,4869f7a5dfa277a7dca6462dcf3b52b2,R$ 249640.70
1535,7c67e1448b00f6e969d365cea6b010ab,R$ 239536.44
1013,53243585a1d6dc2643021fd1853d8905,R$ 235856.68
881,4a3ca9315b744ce9f8e9374361493884,R$ 235539.96
3024,fa1c13f2614d7b5c4749cbc52fecda94,R$ 204084.73
2643,da8622b14eb17ae2831f4ac5b9dab84a,R$ 185192.32
1560,7e93a43ef30c4f03f38b393420bc753a,R$ 182754.05
192,1025f0e2d44d7041d6cf58b6550e0bfa,R$ 172860.69
1505,7a67c85e85bb2ce8582c35f2203ad736,R$ 162648.38
1824,955fee9216a65b617aa5c0531780ce60,R$ 160602.68


Revenue por vendedor TOP 10 id de vendedores que mais venderam

In [ ]:
df_orders_state = pedidos.merge(
    clientes[['customer_id', 'customer_state']],
    on='customer_id'
)
orders_state = df_orders_state.groupby('customer_state')['order_id'].nunique().reset_index()
orders_state.columns = ['estado', 'n_pedidos']
orders_state = orders_state.sort_values('n_pedidos', ascending=False)

display(orders_state)
fig = px.bar(orders_state, x='n_pedidos', y='estado',
             title='Pedidos por Estado', orientation='h')
fig.show()

orders_state.to_csv('pedidos_por_estado.csv', index=False)

,estado,n_pedidos
25,SP,41746
18,RJ,12852
10,MG,11635
22,RS,5466
17,PR,5045
23,SC,3637
4,BA,3380
6,DF,2140
7,ES,2033
8,GO,2020


Numero de Pedidos feito por estado

In [ ]:
df_delay = reviews.merge(
    pedidos[['order_id', 'order_delivered_customer_date', 'order_estimated_delivery_date']],
    on='order_id'
)
df_delay['atraso_dias'] = (df_delay['order_delivered_customer_date'] -
                          df_delay['order_estimated_delivery_date']).dt.days
df_delay = df_delay.dropna(subset=['atraso_dias'])

df_delay['bucket_atraso'] = pd.cut(df_delay['atraso_dias'],
                                  bins=[-10, 0, 3, 7, 14, 30, 100],
                                  labels=['Antecipado', 'No Prazo', '+1-3d', '+4-7d', '+8-14d', '>14d'])

avg_score_delay = df_delay.groupby('bucket_atraso')['review_score'].agg(['mean', 'count']).reset_index()
avg_score_delay = avg_score_delay.sort_values('mean', ascending=False)

display(avg_score_delay.round(2))
fig = px.bar(avg_score_delay, x='mean', y='bucket_atraso',
             title='Avaliação Média por Bucket de Atraso',
             color='mean', orientation='h')
fig.show()

avg_score_delay.to_csv('avaliacao_vs_atraso.csv', index=False)

/tmp/ipython-input-3559447576.py:15: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,bucket_atraso,mean,count
0,Antecipado,4.22,28060
1,No Prazo,3.29,1856
2,+1-3d,2.10,1756
5,>14d,1.94,294
3,+4-7d,1.68,1453
4,+8-14d,1.62,1014


Numero de pedidos feitos por estado

In [ ]:
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=('Revenue Mensal', 'Top Categorias', 'Pedidos por Estado', 'Score vs Atraso'),
                    specs=[[{"type": "scatter"}, {"type": "bar"}],
                           [{"type": "bar"}, {"type": "bar"}]])

fig.add_trace(go.Scatter(x=revenue_month['ano_mes'], y=revenue_month['revenue'], name='Revenue'), row=1, col=1)
fig.add_trace(go.Bar(y=revenue_cat['product_category_name'], x=revenue_cat['revenue'], orientation='h', name='Categorias'), row=1, col=2)
fig.add_trace(go.Bar(y=orders_state['estado'], x=orders_state['n_pedidos'], orientation='h', name='Estados'), row=2, col=1)
fig.add_trace(go.Bar(y=avg_score_delay['bucket_atraso'], x=avg_score_delay['mean'], orientation='h', name='Atraso'), row=2, col=2)

fig.update_layout(height=800, title_text="DASHBOARD OLIST - KPIs Principais")
fig.show()

Resultado Final de todos os Graficos